In [1]:
from sklearn import svm
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

clf = svm.SVC()
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

We now we clf as a trained model. Below this model is saved, loaded and used to predict. This should mimic the use case for our loctree2 model.

In [2]:
from sklearn.externals import joblib
joblib.dump(clf, 'filename.pkl')

['filename.pkl']

In [16]:
clf = joblib.load('filename.pkl') 


Below is the bootstrap algorithm

 Final bootstrap function to be integrated into workflow.

In [14]:
def bootstrap(clf2 ,X_test ,y_test, b):
    '''
    :param X: Input test data
    :param y: test labels
    :param b: number of bootstrap samples to draw (100-1000)
    :return: the standard deviation (std) aver all and per class over all bootstrap samples.
    '''

    #initilize lists to store elements
    #b=100
    
    acc_list = list()
    class_scores = []
    classes = pd.unique(y_test)

    # compute length of data set
    n = len(X_test)

    for i in range(b):

        # generate random indices and draw sample
        X_sub = X_test.sample(int(n*0.6))
        y_sub = y_test[X_sub.index]

        # compute accurace and append to list
        acc = accuracy_score(y_sub, clf2.predict(X_sub))
        acc_list.append(acc)

        # create data frame for per class derivations
        df = pd.DataFrame({'y_pred': clf2.predict(X_sub), 'y': y_sub})
        
        #get number of classes
        
        for j in classes:
            class_scores.append(accuracy_score(df[df.y == j].y, df[df.y == j].y_pred))
    
    # compute standard deviation of accuracys over bootstrap samples
    acc_array = np.array(acc_list)
    sd_acc = np.std(acc_array)

    # convert to numpy for easy calculation
    per_class_acc = np.array(class_scores).reshape(b, len(classes))
    per_class_std = np.std(np.array(class_scores).reshape(b, len(classes)), axis=0)

    return sd_acc, per_class_std

In [15]:
b=100
bootstrap(clf,X,y,b)

(0.0078755756207419536, array([ 0.02407509,  0.02342254,  0.02360387]))